In [1]:
import h5py
import numpy as np

import helpers

In [17]:
sim = 'tng100-3'
base_dir = f'/home/rmcg/data/downloaded/tng/{sim}/'
lhalotree_dir = base_dir + 'merger_tree/lhalotree/'

with h5py.File(lhalotree_dir+'trees_sf1_099.0.hdf5', 'r') as file:
    redshifts = np.array(file['/Header/Redshifts'])
scale_factors = 1 / (1 + redshifts)

<img src="papers/basic_picture.png"/>

## Gas rates
- Use set operations based on particle IDs
- Calculating gas temp can be taken from [here](https://www.tng-project.org/data/forum/topic/338/cold-and-hot-gas/)

<u> Problems </u>
- What happens to particle IDs when gas cells split?
- Calculating R_hot->cold, cells are different masses in prog and sub snapshots


## Star rates
- Use GFM_StellarFormationTime to calculate number of stars formed

<u> Problems </u>
- Stellar particles brought in through mergers aren't accounted for, may mean I underestimate stellar mass
- Stellar masses evolve over time
- Wind particles


In [53]:
def calculate_efficiencies(snap, sub_id, prog_id, diff_dm_mass, stellar_mass):
    
    # Calculations using gas particles
    gas_fields = ['Masses', 'ParticleIDs', 'StarFormationRate']
    prog_g = helpers.loadSubhalo(base_dir, snap-1, prog_id, 0, fields=gas_fields)
    prog_is_cold_gas = prog_g['StarFormationRate'] > 0
    prog_is_hot_gas = np.logical_not(prog_is_cold_gas)
    prog_cold_gas_ids = prog_g['ParticleIDs'][prog_is_cold_gas]
    prog_hot_gas_ids = prog_g['ParticleIDs'][prog_is_hot_gas]
    
    sub_g = helpers.loadSubhalo(base_dir, snap, sub_id, 0, fields=gas_fields)
    sub_is_cold_gas = sub_g['StarFormationRate'] > 0
    sub_is_hot_gas = np.logical_not(sub_is_cold_gas)
    sub_cold_gas_ids = sub_g['ParticleIDs'][sub_is_cold_gas]
    sub_hot_gas_ids = sub_g['ParticleIDs'][sub_is_hot_gas]
    
    rate_hot_cold = 0
    hot_to_cold_ids = set(np.intersect1d(prog_hot_gas_ids, sub_cold_gas_ids))
    for (part_id, mass) in zip(sub_cold_gas_ids, sub_g['Masses'][sub_is_cold_gas]):
        if part_id in hot_to_cold_ids:
            rate_hot_cold += mass
        
    rate_cold_hot = 0
    cold_to_hot_ids = set(np.intersect1d(prog_cold_gas_ids, sub_hot_gas_ids))
    for (part_id, mass) in zip(prog_cold_gas_ids, prog_g['Masses'][prog_is_cold_gas]):
        if part_id in cold_to_hot_ids:
            rate_cold_hot += mass
    
    rate_accrete_hot = 0
    accrete_hot_ids = set(np.setdiff1d(sub_hot_gas_ids, prog_g['ParticleIDs']))
    for (part_id, mass) in zip(sub_hot_gas_ids, sub_g['Masses'][sub_is_hot_gas]):
        if part_id in accrete_hot_ids:
            rate_accrete_hot += mass
    
    cold_gas_mass = np.sum(sub_g['Masses'][sub_is_cold_gas])
    hot_gas_mass = np.sum(sub_g['Masses'][sub_is_hot_gas])
    
    
    # Calculations using star particles
    sub_s = helpers.loadSubhalo(base_dir, snap, subhalo_id, 4,
                                fields=['GFM_StellarFormationTime', 'Masses'])
    assert np.isclose(sub_stellar_mass, np.sum(sub_s['Masses']))
    recently_formed = sub_s['GFM_StellarFormationTime'] > scale_factors[snap-1]
    rate_cold_stars = np.sum(sub_s['Masses'][recently_formed])
    
    # Calculating efficiencies
    f_a = rate_accrete_hot / diff_dm_mass
    f_c = rate_hot_cold / hot_gas_mass
    f_s = rate_cold_stars / cold_gas_mass
    f_d = rate_cold_hot / rate_cold_stars
    
    return f_a, f_c, f_s, f_d

In [72]:
# How much does gas mass change?
sub_g = helpers.loadSubhalo(base_dir, 99, 0, 0, fields=['Masses', 'ParticleIDs'])
prog_g = helpers.loadSubhalo(base_dir, 98, 0, 0, fields=['Masses', 'ParticleIDs'])

sub_ids = sub_g['ParticleIDs']
print(f'Len sub_ids: {sub_ids.shape[0]}')
prog_ids = prog_g['ParticleIDs']
print(f'Len prog_ids: {prog_ids.shape[0]}')
intersect = np.intersect1d(sub_ids, prog_ids)
print(f'Len intersect: {intersect.shape[0]}')

for part_id in intersect[:10]:
    sub_mass = sub_g['Masses'][sub_g['ParticleIDs'] == part_id][0]
    prog_mass = prog_g['Masses'][prog_g['ParticleIDs'] == part_id][0]
    print(f'Prog mass: {prog_mass:.3g}, sub mass: {sub_mass:.3g}, increase: {sub_mass > prog_mass}')

Len sub_ids: 696413
Len prog_ids: 681986
Len intersect: 652293
Prog mass: 0.00605, sub mass: 0.00601, increase: False
Prog mass: 0.00556, sub mass: 0.00567, increase: True
Prog mass: 0.00637, sub mass: 0.00666, increase: True
Prog mass: 0.00622, sub mass: 0.00575, increase: False
Prog mass: 0.0093, sub mass: 0.00885, increase: False
Prog mass: 0.00598, sub mass: 0.006, increase: True
Prog mass: 0.00508, sub mass: 0.00497, increase: False
Prog mass: 0.00595, sub mass: 0.00554, increase: False
Prog mass: 0.00513, sub mass: 0.00457, increase: False
Prog mass: 0.00755, sub mass: 0.00673, increase: False
